In [26]:
import pandas as pd
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', 1000)

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from datetime import timezone, datetime
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder
#from sklearn.compose import ColumnTransformer 
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold 
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import classification_report, confusion_matrix

import itertools

In [27]:
drg = pd.read_csv('../data/DRGCODES.csv')
diabetes = pd.read_csv('../data/Diabetes_Numerical.csv')
patients = pd.read_csv('../data/PATIENTS.csv')

In [31]:
drg

,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,342,2491,144486,HCFA,28,"TRAUMATIC STUPOR & COMA, COMA <1 HR AGE >17 WI...",NaN,NaN
1,343,24958,162910,HCFA,110,MAJOR CARDIOVASCULAR PROCEDURES WITH COMPLICAT...,NaN,NaN
2,344,18325,153751,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN
3,345,17887,182692,HCFA,14,SPECIFIC CEREBROVASCULAR DISORDERS EXCEPT TRAN...,NaN,NaN
4,346,11113,157980,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN
...,...,...,...,...,...,...,...,...
125552,123452,71582,101422,MS,221,CARDIAC VALVE & OTH MAJ CARDIOTHORACIC PROC W/...,NaN,NaN
125553,123453,46449,110075,APR,1653,Coronary Bypass w/ Cardiac Cath Or Percutaneou...,3.0,2.0
125554,123454,46449,110075,APR,1653,Coronary Bypass w/ Cardiac Cath Or Percutaneou...,3.0,2.0
125555,123455,46449,110075,MS,234,CORONARY BYPASS W CARDIAC CATH W/O MCC,NaN,NaN


In [30]:
drg3 = drg.dropna()
drg3

,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
10546,13782,21729,192719,APR,44,Tracheostomy W Long Term Mechanical Ventilatio...,4.0,3.0
10548,13784,9270,171336,APR,44,Tracheostomy W Long Term Mechanical Ventilatio...,4.0,3.0
10560,13796,11318,151558,APR,7204,Septicemia & Disseminated Infections,4.0,4.0
10713,15353,252,190159,APR,2804,Alcoholic Liver Disease,4.0,4.0
10714,15354,25188,160667,APR,1694,Major Thoracic & Abdominal Vascular Procedures,4.0,3.0
...,...,...,...,...,...,...,...,...
125550,123450,71582,101422,APR,1632,Cardiac Valve Procedures w/o Cardiac Catheteri...,2.0,1.0
125551,123451,71582,101422,APR,1632,Cardiac Valve Procedures w/o Cardiac Catheteri...,2.0,1.0
125553,123453,46449,110075,APR,1653,Coronary Bypass w/ Cardiac Cath Or Percutaneou...,3.0,2.0
125554,123454,46449,110075,APR,1653,Coronary Bypass w/ Cardiac Cath Or Percutaneou...,3.0,2.0


In [35]:
patientgroup= drg3.groupby("SUBJECT_ID")
patientgroup
severity= patientgroup['DRG_SEVERITY'].max()
severity
mortality= patientgroup['DRG_MORTALITY'].max()
mortality

SUBJECT_ID
2        1.0
7        1.0
11       1.0
17       3.0
19       3.0
        ... 
99985    4.0
99991    4.0
99992    4.0
99995    3.0
99999    1.0
Name: DRG_MORTALITY, Length: 31132, dtype: float64

In [37]:
diabetesdrg = pd.merge(diabetes, severity, how='inner', on='SUBJECT_ID' )
diabetesdrg = pd.merge(diabetesdrg, mortality, how='inner', on='SUBJECT_ID')
diabetesdrg



,SUBJECT_ID,LOS,HOSPITALIZATION,ORDINAL_AGE,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government,Medicaid,Medicare,Private,Self Pay,DIVORCED,LIFE PARTNER,MARRIED,NAN,SEPARATED,SINGLE,UNKNOWN,WIDOWED,GENDER,3,4,5,7,8,9,10,11,12,13,14,18,27,31,32,34,35,38,39,40,41,42,46,47,48,49,52,53,54,57,58,66,70,75,78,79,82,86,88,94,96,97,110,111,112,117,120,121,127,130,131,135,136,137,138,139,140,141,142,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,161,162,163,164,171,172,173,174,175,176,180,182,183,184,185,186,187,188,189,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,207,208,209,210,211,212,213,214,215,216,217,218,220,221,223,225,226,227,228,229,230,232,233,235,236,237,238,239,240,241,242,244,245,246,249,250,251,252,253,254,255,256,257,258,259,261,262,263,265,266,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,303,304,305,306,307,308,309,310,311,312,313,314,315,317,318,319,320,321,322,323,324,325,327,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,382,383,385,386,387,388,389,390,391,394,395,396,397,398,401,402,403,404,405,410,411,412,413,414,415,416,417,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,440,441,442,443,444,445,446,447,448,449,451,452,453,454,455,456,457,458,459,461,462,463,464,465,466,471,472,473,475,477,478,480,481,482,483,484,485,486,487,488,490,491,492,493,494,495,496,500,501,502,506,507,508,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,540,541,542,543,550,551,552,553,555,556,557,558,560,562,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,607,608,611,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,632,633,634,642,644,646,647,648,649,652,654,666,670,674,680,681,682,683,684,685,686,690,692,693,694,695,696,697,698,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,741,742,745,746,747,748,750,751,752,753,754,755,756,757,758,759,765,766,770,774,775,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,799,800,801,802,803,804,805,806,807,808,810,811,812,813,814,815,816,817,820,821,822,823,824,825,826,830,831,833,834,835,836,839,840,842,845,846,847,850,851,852,853,854,860,861,862,863,864,865,866,867,868,869,870,872,873,874,875,879,880,881,882,883,884,886,887,890,891,892,893,894,896,897,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,916,917,918,919,920,921,922,923,924,926,927,933,934,935,936,937,939,941,942,944,945,947,948,950,951,952,953,954,955,956,958,959,961,962,964,965,966,967,968,969,970,971,972,974,975,977,983,987,989,990,991,992,994,995,996,997,998,999,E812,E813,E814,E815,E816,E817,E818,E819,E821,E822,E823,E824,E825,E826,E838,E848,E849,E850,E851,E852,E853,E854,E855,E857,E858,E866,E869,E870,E871,E874,E876,E878,E879,E880,E881,E882,E884,E885,E887,E888,E890,E899,E900,E901,E906,E910,E911,E912,E915,E916,E917,E918,E920,E922,E924,E927,E928,E929,E930,E931,E932,E933,E934,E935,E936,E937,E938,E939,E940,E941,E942,E943,E944,E945,E946,E947,E950,E953,E955,E956,E957,E958,E960,E965,E966,E967,E968,E969,E980,E988,E989,V01,V02,V03,V04,V05,V06,V07,V08,V09,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V23,V26,V27,V29,V30,V42,V43,V44,V45,V46,V49,V50,V53,V54,V55,V56,V58,V60,V61,V62,V63,V64,V65,V66,V69,V70,V72,V74,V76,DEAD,ASIAN,WHITE,UNKNOWN/NOT SPECIFIED,BLACK/AFRICAN AMERICAN,PATIENT DECLINED TO ANSWER,OTHER,HISPANIC OR LATINO,HISPANIC/LATINO - GUATEMALAN,ASIAN - CHINESE,HISPANIC/LATINO - PUERTO RICAN,ASIAN - ASIAN INDIAN,ASIAN - VIETNAMESE,MULTI RACE ETHNICITY,HISPANIC/LATINO - DOMINICAN,AMERICAN INDIAN/ALASKA NATIVE,WHITE - RUSSIAN,BLACK/

In [38]:
#diabetesdrg.to_csv(r'../data/diabetesdrg.csv', index = False)

In [25]:
#attempt2
#add DRG serverity and mortality to diabetes matrix
#take the max drg severity and mortality for each patient, nan val?
drgnona = drg.fillna(0)
drgnona.tail(10)

patientgroup= drgnona.groupby("SUBJECT_ID")
patientgroup.head()
severity= patientgroup['DRG_SEVERITY'].max()
severity.head(40)



SUBJECT_ID
2     2.0
3     0.0
4     0.0
5     0.0
6     0.0
7     2.0
8     0.0
9     0.0
10    0.0
11    1.0
12    0.0
13    0.0
16    0.0
17    3.0
18    0.0
19    3.0
20    0.0
21    4.0
22    0.0
23    2.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
30    0.0
31    0.0
32    0.0
33    0.0
34    3.0
35    3.0
36    3.0
37    0.0
38    4.0
39    0.0
41    4.0
42    0.0
43    3.0
44    0.0
45    0.0
Name: DRG_SEVERITY, dtype: float64

In [20]:
patientgroup.head(10)

,ROW_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,342,144486,HCFA,28,"TRAUMATIC STUPOR & COMA, COMA <1 HR AGE >17 WI...",0.0,0.0
1,343,162910,HCFA,110,MAJOR CARDIOVASCULAR PROCEDURES WITH COMPLICAT...,0.0,0.0
2,344,153751,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,0.0,0.0
3,345,182692,HCFA,14,SPECIFIC CEREBROVASCULAR DISORDERS EXCEPT TRAN...,0.0,0.0
4,346,157980,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,0.0,0.0
5,347,154221,HCFA,385,"NEONATES, DIED OR TRANSFERRED TO ANOTHER ACUTE...",0.0,0.0
6,348,122380,HCFA,105,CARDIAC VALVE & OTHER MAJOR CARDIOTHORACIC PRO...,0.0,0.0
7,349,194891,HCFA,101,OTHER RESPIRATORY SYSTEM DIAGNOSES WITH COMPLI...,0.0,0.0
8,350,171485,HCFA,107,CORONARY BYPASS WITH CARDIAC CATHETER,0.0,0.0
9,351,164767,HCFA,14,SPECIFIC CEREBROVASCULAR DISORDERS EXCEPT TRAN...,0.0,0.0


In [ ]:
severity= patientgroup['DRG_SEVERITY'].max()
severity.head(40)

In [ ]:
severity.loc[severity['DRG_SEVERITY'] == 0]

In [13]:
#number of null rows
severity.isnull().sum(axis = 0)

15378

In [24]:
uniquesid = drg['SUBJECT_ID'].unique()
len(uniquesid)

46511

In [25]:
drg.groupby('SUBJECT_ID').nunique()

,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
SUBJECT_ID,,,,,,,,
2,2,1,1,2,2,2,1,1
3,1,1,1,1,1,1,0,0
4,1,1,1,1,1,1,0,0
5,1,1,1,1,1,1,0,0
6,1,1,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...
99985,3,1,1,2,2,2,1,1
99991,3,1,1,2,2,2,1,1
99992,3,1,1,2,2,2,1,1


In [26]:
drg.loc[drg['SUBJECT_ID'] == 99999]

,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
79324,78340,99999,113369,APR,3042,Dorsal & Lumbar Fusion Proc Except for Curvatu...,2.0,1.0
79325,78341,99999,113369,APR,3042,Dorsal & Lumbar Fusion Proc Except for Curvatu...,2.0,1.0
79326,78342,99999,113369,MS,460,SPINAL FUSION EXCEPT CERVICAL W/O MCC,NaN,NaN


In [27]:
patients.loc[patients['SUBJECT_ID'] == 99999]

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
41095,46520,99999,F,2054-01-06 00:00:00,NaN,NaN,NaN,0


In [28]:
uniquesid2 = patients['SUBJECT_ID'].unique()
len(uniquesid2)

46520